In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

plt.style.use('default')
# Set options to display all columns and expand column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Set options to display a specific number of columns and control column width
pd.set_option('display.max_columns', 10)  # Set the maximum number of columns to display
pd.set_option('display.max_colwidth', 100)  # Set the maximum column width for text data
pd.set_option('display.max_rows', 100)  # Set the maximum number of columns to display

In [ ]:

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (4, 4)
# Set the style of the plot for publication
#sns.set(style="white", context="talk")  # 'talk' context increases font sizes


In [ ]:
pip list

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu

# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
cell_types = ["Epithelial"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

metadata = pd.read_csv(metadat_path)
metadat = metadata

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())

# Filter out samples with too many NaNs
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)


samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index


combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)


# Calculate the percentage of significant features per sample for each condition
sig_percentage_cond1 = calculate_significant_percentage(combined_pval_cond1, p_value_threshold)



cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())


nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)


samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index


combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)

# Calculate the percentage of significant features per sample for each condition

sig_percentage_cond2 = calculate_significant_percentage(combined_pval_cond2, p_value_threshold)



# Perform a Mann-Whitney U test to compare the two conditions
stat, p_value = mannwhitneyu(sig_percentage_cond1, sig_percentage_cond2, alternative='two-sided')
print(f"Mann-Whitney U test p-value: {p_value:.4f}")

# Prepare data for plotting
plot_data = pd.DataFrame({
    'Condition': ['Epithelial'] * len(sig_percentage_cond1) + ['Cancer'] * len(sig_percentage_cond2),
    'Percentage of Significant Features': np.concatenate([sig_percentage_cond1.values, sig_percentage_cond2.values])
})

# Plot a boxplot comparing the two conditions
plt.figure(figsize=(2.5, 6))
sns.boxplot(x='Condition', y='Percentage of Significant Features', data=plot_data)
sns.stripplot(x='Condition', y='Percentage of Significant Features', data=plot_data, color='black', alpha=0.5, jitter=False)
plt.title(f'Comparison of Significant Feature Percentages Between LUAD and LUSC Conditions\nMann-Whitney U test p-value: {p_value:.4f}')
plt.ylabel('Percentage of Significant Features (%)')
plt.xlabel('Condition')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu

# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
cell_types = ["Epithelial"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

metadata = pd.read_csv(metadat_path)
metadat = metadata

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())

# Filter out samples with too many NaNs
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)


samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index


combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)


# Calculate the percentage of significant features per sample for each condition
sig_percentage_cond1 = calculate_significant_percentage(combined_pval_cond1, p_value_threshold)



cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())


nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)


samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index


combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)

# Calculate the percentage of significant features per sample for each condition

sig_percentage_cond2 = calculate_significant_percentage(combined_pval_cond2, p_value_threshold)



# Perform a Mann-Whitney U test to compare the two conditions
stat, p_value = mannwhitneyu(sig_percentage_cond1, sig_percentage_cond2, alternative='two-sided')
print(f"Mann-Whitney U test p-value: {p_value:.4f}")

# Prepare data for plotting
plot_data = pd.DataFrame({
    'Condition': ['Epithelial'] * len(sig_percentage_cond1) + ['Cancer'] * len(sig_percentage_cond2),
    'Percentage of Significant Features': np.concatenate([sig_percentage_cond1.values, sig_percentage_cond2.values])
})

# Plot a boxplot comparing the two conditions
plt.figure(figsize=(2.5, 6))
sns.boxplot(x='Condition', y='Percentage of Significant Features', data=plot_data)
sns.stripplot(x='Condition', y='Percentage of Significant Features', data=plot_data, color='black', alpha=0.5, jitter=False)
plt.title(f'Comparison of Significant Feature Percentages Between LUAD and LUSC Conditions\nMann-Whitney U test p-value: {p_value:.4f}')
plt.ylabel('Percentage of Significant Features (%)')
plt.xlabel('Condition')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu

# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
cell_types = ["Epithelial"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

metadata = pd.read_csv(metadat_path)
metadat = metadata

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())

# Filter out samples with too many NaNs
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)


samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index


combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)


# Calculate the percentage of significant features per sample for each condition
sig_percentage_cond1 = calculate_significant_percentage(combined_pval_cond1, p_value_threshold)



cell_types = ["Immune"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())


nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)


samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index


combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)

# Calculate the percentage of significant features per sample for each condition

sig_percentage_cond2 = calculate_significant_percentage(combined_pval_cond2, p_value_threshold)



# Perform a Mann-Whitney U test to compare the two conditions
stat, p_value = mannwhitneyu(sig_percentage_cond1, sig_percentage_cond2, alternative='two-sided')
print(f"Mann-Whitney U test p-value: {p_value:.4f}")

# Prepare data for plotting
plot_data = pd.DataFrame({
    'Condition': ['Epithelial'] * len(sig_percentage_cond1) + ['Immune'] * len(sig_percentage_cond2),
    'Percentage of Significant Features': np.concatenate([sig_percentage_cond1.values, sig_percentage_cond2.values])
})

# Plot a boxplot comparing the two conditions
plt.figure(figsize=(2.5, 6))
sns.boxplot(x='Condition', y='Percentage of Significant Features', data=plot_data)
sns.stripplot(x='Condition', y='Percentage of Significant Features', data=plot_data, color='black', alpha=0.5, jitter=False)
plt.title(f'Comparison of Significant Feature Percentages Between LUAD and LUSC Conditions\nMann-Whitney U test p-value: {p_value:.4f}')
plt.ylabel('Percentage of Significant Features (%)')
plt.xlabel('Condition')
plt.show()



In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
#cell_types=["Bcell", "cDC1", "cDC2", "DCmature", "Macrophage", "Macrophagealveolar", "Mastcell", "Monocyte", "NKcell", "pDC", "Plasmacell", "TcellCD4", "TcellCD8", "Tcellregulatory"]

#cell_types=("B cell", "cDC2", "Macrophage", "Macrophage alveolar", "Mast cell", "Monocyte", "NK cell", "pDC", "Plasma cell", "T cell CD4", "T cell CD8", "T cell regulatory")

#cell_types = ["Macrophage alveolar"]

cell_types=["Cancer"]

nan_threshold = 800

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['uicc_stage'] == "IV")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['tumor_stage'] == "advanced")]['patient'].unique()
#unique_samples = metadata['patient'].unique()

#yes_list = ["EGRF", "EGFR", "EGFR, TP53", "EGFR, KMT2D, U2AF1, RET", "EGRF, TP53"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    print(cell_type)
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]

    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)

    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')


#we already filter when loading the cell type specific dfs. But we want to get rid of the empty samples in the combined
#dfs as well. Make sure you don't remove samples you want to keep when running several cell types in the loop together
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)


# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Handle NA values in p-values
#combined_r2.fillna(1, inplace=True)


print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
#print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

In [ ]:
sorted_df.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/Cancer_hallmarks_R1.csv")

sorted_df[:100]

In [ ]:
top_common_features = sorted_df["GeneSet"][:200]
top_common_features

In [ ]:
####LUAD vs LUSC comparison


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu

# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

metadata = pd.read_csv(metadat_path)
metadat = metadata

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1) & (metadata['condition'] == "LUAD")]["patient"].tolist())

# Filter out samples with too many NaNs
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)


samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index


combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)


# Calculate the percentage of significant features per sample for each condition
sig_percentage_cond1 = calculate_significant_percentage(combined_pval_cond1, p_value_threshold)



cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

p_value_threshold = 0.05  # Define the significance threshold for p-values

# Function to process data for samples based on condition
def process_condition(metadat, unique_samples):
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to calculate percentage of significant features per sample
def calculate_significant_percentage(pval_df, threshold=0.05):
    # Calculate the number of significant features per sample
    sig_counts = (pval_df < threshold).sum(axis=1, skipna=True)
    # Calculate the number of available features per sample (non-NaN)
    available_features = pval_df.notna().sum(axis=1)
    # Calculate the percentage of significant features
    significant_percentage = (sig_counts / available_features) * 100
    return significant_percentage

# Process each condition
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['tissue: tumor_primary'] == 1) & (metadata['condition'] == "LUSC")]["patient"].tolist())


nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)


samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index


combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)

# Calculate the percentage of significant features per sample for each condition

sig_percentage_cond2 = calculate_significant_percentage(combined_pval_cond2, p_value_threshold)



# Perform a Mann-Whitney U test to compare the two conditions
stat, p_value = mannwhitneyu(sig_percentage_cond1, sig_percentage_cond2, alternative='two-sided')
print(f"Mann-Whitney U test p-value: {p_value:.4f}")

# Prepare data for plotting
plot_data = pd.DataFrame({
    'Condition': ['LUAD'] * len(sig_percentage_cond1) + ['LUSC'] * len(sig_percentage_cond2),
    'Percentage of Significant Features': np.concatenate([sig_percentage_cond1.values, sig_percentage_cond2.values])
})

# Plot a boxplot comparing the two conditions
plt.figure(figsize=(2.5, 6))
sns.boxplot(x='Condition', y='Percentage of Significant Features', data=plot_data, palette = ["pink", "green"])
sns.stripplot(x='Condition', y='Percentage of Significant Features', data=plot_data, color='black', alpha=0.5, jitter=False)
plt.title(f'Comparison of Significant Feature Percentages Between LUAD and LUSC Conditions\nMann-Whitney U test p-value: {p_value:.4f}')
plt.ylabel('Percentage of Significant Features (%)')
plt.xlabel('Condition')
plt.show()

In [ ]:
import pandas as pd
import os

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

nan_threshold = 800

metadat = metadata

# Define the cell types and the base paths
#cell_types = ["Cancer", "Stromal", "Endothelial"]
cell_types = ["Cancer"]


# Function to read and process data for samples based on GFR threshold
def process_condition(metadat, unique_samples):

    
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to count significance
def count_significance(pval_df, threshold=0.05):
    sig_count = (pval_df < threshold).sum(axis=0, skipna=True)
    nonsig_count = (pval_df >= threshold).sum(axis=0, skipna=True)
    return sig_count, nonsig_count

# Read metadata

#metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()

# Process each condition based on GFR threshold
#combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD")]["patient"].tolist())  # Samples above the threshold
#combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]["patient"].tolist())  # Samples below the threshold

combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())  
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())  


#filter out the samples with too many nan
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)
nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index
samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index

combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)
combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)


# Count significance for each condition
sig_count_cond1, nonsig_count_cond1 = count_significance(combined_pval_cond1)
sig_count_cond2, nonsig_count_cond2 = count_significance(combined_pval_cond2)

# Creating the comparison DataFrame
comparison_df = pd.DataFrame(columns=combined_pval_cond1.columns, index=['cond1_sig', 'cond1_nonsig', 'cond2_sig', 'cond2_nonsig'])
comparison_df.loc['cond1_sig'] = sig_count_cond1
comparison_df.loc['cond1_nonsig'] = nonsig_count_cond1
comparison_df.loc['cond2_sig'] = sig_count_cond2
comparison_df.loc['cond2_nonsig'] = nonsig_count_cond2


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

# Assuming the comparison_df has been generated as in the previous code

fisher_results = pd.DataFrame(columns=['Feature', 'Test_statistic', 'p_value', "odds_ratio", 'cond1_sig_count', 'cond2_sig_count'])

for feature in comparison_df.columns:
    # Creating the contingency table for each feature
    contingency_table = comparison_df[[feature]].values.reshape(2, 2)
    
    # Extract counts of significant samples for each condition for the current feature
    cond1_sig_count = comparison_df.loc['cond1_sig', feature]
    cond2_sig_count = comparison_df.loc['cond2_sig', feature]

    # Apply Fisher's Exact Test
    odds_ratio, p = fisher_exact(contingency_table)
    
    # Store the results
    result_row = pd.DataFrame([{
        'Feature': feature, 
        'Test_statistic': odds_ratio, 
        'p_value': p, 
        "odds_ratio": odds_ratio, 
        'cond1_sig_count': cond1_sig_count,
        'cond2_sig_count': cond2_sig_count
    }])
    
    fisher_results = pd.concat([fisher_results, result_row], ignore_index=True)

# Sort the results by p-value in ascending order
sorted_results = fisher_results.sort_values(by='p_value')

# Display the most significant features
print(sorted_results.head())  # Adjust the number inside head() as needed

from statsmodels.stats.multitest import multipletests
# Extract the original p-values
p_values = sorted_results['p_value'].values

# Apply multiple test correction (using Benjamini-Hochberg method as an example)
correction_method = 'fdr_bh'  # You can change this to 'bonferroni', 'holm', etc., based on your requirement
rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method=correction_method)

# Add the corrected p-values to your DataFrame
sorted_results['corrected_p_value'] = corrected_p_values
sorted_results['rejected_h0'] = rejected

# Now sorted_results has an additional column with corrected p-values
print(sorted_results.head())  # Adjust the number inside head() as needed

sorted_results.head(100)

In [ ]:
sorted_results["Feature"].head(100)

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]

#"carcinoma" related pathways

top_common_features = [
    "Cancer_REACTOME_MET_ACTIVATES_PTK2_SIGNALING",
    "Cancer_PID_EPHRINB_REV_PATHWAY",
    "Cancer_REACTOME_CONSTITUTIVE_SIGNALING_BY_ABERRANT_PI3K_IN_CANCER",
    "Cancer_KEGG_MEDICUS_REFERENCE_ITGA_B_FAK_RAC_SIGNALING_PATHWAY",
    "Cancer_PID_INTEGRIN5_PATHWAY",
     "Cancer_KEGG_PYRIMIDINE_METABOLISM",
    "Cancer_REACTOME_DETOXIFICATION_OF_REACTIVE_OXYGEN_SPECIES",
    "Cancer_WP_ELECTRON_TRANSPORT_CHAIN_OXPHOS_SYSTEM_IN_MITOCHONDRIA",
    "Cancer_PID_TOLL_ENDOGENOUS_PATHWAY",
    "Cancer_WP_P53_TRANSCRIPTIONAL_GENE_NETWORK",
   #"Cancer_BIOCARTA_VEGF_PATHWAY",
    "Cancer_REACTOME_GLUTATHIONE_CONJUGATION",
      #"Cancer_KEGG_VEGF_SignalingPATHWAY",
    "Cancer_REACTOME_SYNTHESIS_OF_BILE_ACIDS_AND_BILE_SALTS",
    "Cancer_KEGG_STEROID_HORMONE_BIOSYNTHESIS",
    "Cancer_REACTOME_SIGNALING_BY_CSF3_G_CSF",
    #"Cancer_REACTOME_PEPTIDE_HORMONE_METABOLISM",
    
]

#top_common_features = list(set(top_common_features))


cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])



# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ['Goveia_Carmeliet_2020_patient_45',
 'Goveia_Carmeliet_2020_patient_46',
 'Goveia_Carmeliet_2020_patient_50',
 'He_Fan_2021_P1',
 'He_Fan_2021_P2',
 'He_Fan_2021_P3',
 'He_Fan_2021_P4',
 'He_Fan_2021_P5',
 'Kim_Lee_2020_P0006',
 'Kim_Lee_2020_P0008',
 'Kim_Lee_2020_P0009',
 'Kim_Lee_2020_P0018',
 'Kim_Lee_2020_P0019',
 'Kim_Lee_2020_P0020',
 'Kim_Lee_2020_P0025',
 'Kim_Lee_2020_P0028',
 'Kim_Lee_2020_P0030',
 'Kim_Lee_2020_P0031',
 'Kim_Lee_2020_P0034',
 'Kim_Lee_2020_P1006',
 'Kim_Lee_2020_P1028',
 'Kim_Lee_2020_P1049',
 'Kim_Lee_2020_P1058',
 'Lambrechts_Thienpont_2018_6149v2_3',
 'Lambrechts_Thienpont_2018_6149v2_4',
 'Lambrechts_Thienpont_2018_6653_6',
 'Laughney_Massague_2020_LX653',
 'Laughney_Massague_2020_LX661',
 'Laughney_Massague_2020_LX675',
 'Laughney_Massague_2020_LX676',
 'Laughney_Massague_2020_LX679',
 'Laughney_Massague_2020_LX680',
 'Laughney_Massague_2020_LX682',
 'Laughney_Massague_2020_LX684',
 'Leader_Merad_2021_338',
 'Leader_Merad_2021_370',
 'Leader_Merad_2021_378',
 'Leader_Merad_2021_393',
 'Leader_Merad_2021_403',
 'Leader_Merad_2021_408',
 'Leader_Merad_2021_410',
 'Leader_Merad_2021_514',
 'Leader_Merad_2021_558',
 'Leader_Merad_2021_564',
 'Leader_Merad_2021_569',
 'Leader_Merad_2021_571',
 'Leader_Merad_2021_572',
 'Leader_Merad_2021_578',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_729',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_593',
 'Leader_Merad_2021_596',
 'Leader_Merad_2021_626',
 'Leader_Merad_2021_630',
 'Leader_Merad_2021_695',
 'Leader_Merad_2021_458',
 'Leader_Merad_2021_460',
 'Leader_Merad_2021_464',
 'Leader_Merad_2021_532',
 'Leader_Merad_2021_725',
 'Leader_Merad_2021_800',
 'Leader_Merad_2021_695',
 'Goveia_Carmeliet_2020_patient_32',
 'Goveia_Carmeliet_2020_patient_40',
 'Goveia_Carmeliet_2020_patient_41',
 'Goveia_Carmeliet_2020_patient_42',
 'Lambrechts_Thienpont_2018_6149v1_1',
 'Lambrechts_Thienpont_2018_6149v1_2',
 'Lambrechts_Thienpont_2018_6653_7',
 'Leader_Merad_2021_377',
 'Leader_Merad_2021_522',
 'Leader_Merad_2021_714',
 'Leader_Merad_2021_584',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_522'
       ]

custom_order = [sample for sample in custom_order if sample in plotting_matrix.index]


# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

plotting_matrix.columns = plotting_matrix.columns.str.replace('^Cancer_', '', regex=True)

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(24, 14), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
#cell_types=["Bcell", "cDC1", "cDC2", "DCmature", "Macrophage", "Macrophagealveolar", "Mastcell", "Monocyte", "NKcell", "pDC", "Plasmacell", "TcellCD4", "TcellCD8", "Tcellregulatory"]

#cell_types=("B cell", "cDC2", "Macrophage", "Macrophage alveolar", "Mast cell", "Monocyte", "NK cell", "pDC", "Plasma cell", "T cell CD4", "T cell CD8", "T cell regulatory")

#cell_types = ["Macrophage alveolar"]

cell_types=["Cancer"]

nan_threshold = 800

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_nonTumorRef_Primary"
r2_folder = "R2_C2_0_0/"
pval_folder = "Padj_C2_0_0/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['uicc_stage'] == "IV")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['tumor_stage'] == "advanced")]['patient'].unique()
#unique_samples = metadata['patient'].unique()

#yes_list = ["EGRF", "EGFR", "EGFR, TP53", "EGFR, KMT2D, U2AF1, RET", "EGRF, TP53"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'].isin(["LUAD", "LUSC"]))]['patient'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    print(cell_type)
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]

    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)

    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')


#we already filter when loading the cell type specific dfs. But we want to get rid of the empty samples in the combined
#dfs as well. Make sure you don't remove samples you want to keep when running several cell types in the loop together
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)


# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Handle NA values in p-values
#combined_r2.fillna(1, inplace=True)


print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
#print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)



In [ ]:
sorted_df.head(50)

In [ ]:
custom_order

In [ ]:
metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'].isin(["LUSC"]))]['patient'].unique()

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]
#cell_types=["Bcell", "cDC1", "cDC2", "DCmature", "Macrophage", "Macrophagealveolar", "Mastcell", "Monocyte", "NKcell", "pDC", "Plasmacell", "TcellCD4", "TcellCD8", "Tcellregulatory"]

#cell_types=("B cell", "cDC2", "Macrophage", "Macrophage alveolar", "Mast cell", "Monocyte", "NK cell", "pDC", "Plasma cell", "T cell CD4", "T cell CD8", "T cell regulatory")

#cell_types = ["Macrophage alveolar"]

cell_types=["Cancer"]

nan_threshold = 2000

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/nonTumorRef_GSEA"
r2_folder = "Pval/"
pval_folder = "Pval/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['uicc_stage'] == "IV")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['tumor_stage'] == "advanced")]['patient'].unique()
#unique_samples = metadata['patient'].unique()

#yes_list = ["EGRF", "EGFR", "EGFR, TP53", "EGFR, KMT2D, U2AF1, RET", "EGRF, TP53"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'].isin(["LUAD", "LUSC"]))]['patient'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    print(cell_type)
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]

    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)

    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')


#we already filter when loading the cell type specific dfs. But we want to get rid of the empty samples in the combined
#dfs as well. Make sure you don't remove samples you want to keep when running several cell types in the loop together
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)


# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Handle NA values in p-values
#combined_r2.fillna(1, inplace=True)


print(combined_r2)

# Initialize a MinMaxScaler
min_max_scaler = MinMaxScaler()

# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = min_max_scaler.fit_transform(combined_r2.values)

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
#print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

In [ ]:
sorted_df.head(100)

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]

#"carcinoma" related pathways

top_common_features = ["Cancer_WP_CHOLESTEROL_BIOSYNTHESIS_PATHWAY",
                       "Cancer_WP_SMITH_MAGENIS_AND_POTOCKI_LUPSKI_SYNDROME_COPY_NUMBER_VARIATION",
    "Cancer_REACTOME_DISEASES_ASSOCIATED_WITH_SURFACTANT_METABOLISM",
                       "Cancer_WP_PATHWAYS_AFFECTED_IN_ADENOID_CYSTIC_CARCINOMA",
                       "Cancer_WP_TH17_CELL_DIFFERENTIATION_PATHWAY",
                       "Cancer_REACTOME_INTERLEUKIN_6_FAMILY_SIGNALING",
                       "Cancer_KEGG_ASTHMA",
                        "Cancer_KEGG_ALLOGRAFT_REJECTION", 
                       
                      ]

#top_common_features = list(set(top_common_features))


cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 2000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/nonTumorRef_GSEA"
r2_folder = "Pval/"
pval_folder = "Pval/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Set p-value threshold
p_value_threshold = 0.25


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 0, combined_r2[col])



# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ['Goveia_Carmeliet_2020_patient_45',
 'Goveia_Carmeliet_2020_patient_46',
 'Goveia_Carmeliet_2020_patient_50',
 'He_Fan_2021_P1',
 'He_Fan_2021_P2',
 'He_Fan_2021_P3',
 'He_Fan_2021_P4',
 'He_Fan_2021_P5',
 'Kim_Lee_2020_P0006',
 'Kim_Lee_2020_P0008',
 'Kim_Lee_2020_P0009',
 'Kim_Lee_2020_P0018',
 'Kim_Lee_2020_P0019',
 'Kim_Lee_2020_P0020',
 'Kim_Lee_2020_P0025',
 'Kim_Lee_2020_P0028',
 'Kim_Lee_2020_P0030',
 'Kim_Lee_2020_P0031',
 'Kim_Lee_2020_P0034',
 'Kim_Lee_2020_P1006',
 'Kim_Lee_2020_P1028',
 'Kim_Lee_2020_P1049',
 'Kim_Lee_2020_P1058',
 'Lambrechts_Thienpont_2018_6149v2_3',
 'Lambrechts_Thienpont_2018_6149v2_4',
 'Lambrechts_Thienpont_2018_6653_6',
 'Laughney_Massague_2020_LX653',
 'Laughney_Massague_2020_LX661',
 'Laughney_Massague_2020_LX675',
 'Laughney_Massague_2020_LX676',
 'Laughney_Massague_2020_LX679',
 'Laughney_Massague_2020_LX680',
 'Laughney_Massague_2020_LX682',
 'Laughney_Massague_2020_LX684',
 'Leader_Merad_2021_338',
 'Leader_Merad_2021_370',
 'Leader_Merad_2021_378',
 'Leader_Merad_2021_393',
 'Leader_Merad_2021_403',
 'Leader_Merad_2021_408',
 'Leader_Merad_2021_410',
 'Leader_Merad_2021_514',
 'Leader_Merad_2021_558',
 'Leader_Merad_2021_564',
 'Leader_Merad_2021_569',
 'Leader_Merad_2021_571',
 'Leader_Merad_2021_572',
 'Leader_Merad_2021_578',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_729',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_593',
 'Leader_Merad_2021_596',
 'Leader_Merad_2021_626',
 'Leader_Merad_2021_630',
 'Leader_Merad_2021_695',
 'Leader_Merad_2021_458',
 'Leader_Merad_2021_460',
 'Leader_Merad_2021_464',
 'Leader_Merad_2021_532',
 'Leader_Merad_2021_725',
 'Leader_Merad_2021_800',
 'Leader_Merad_2021_695',
 'Goveia_Carmeliet_2020_patient_32',
 'Goveia_Carmeliet_2020_patient_40',
 'Goveia_Carmeliet_2020_patient_41',
 'Goveia_Carmeliet_2020_patient_42',
 'Lambrechts_Thienpont_2018_6149v1_1',
 'Lambrechts_Thienpont_2018_6149v1_2',
 'Lambrechts_Thienpont_2018_6653_7',
 'Leader_Merad_2021_377',
 'Leader_Merad_2021_522',
 'Leader_Merad_2021_714',
 'Leader_Merad_2021_584',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_522'
       ]

custom_order = [sample for sample in custom_order if sample in plotting_matrix.index]


# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

plotting_matrix.columns = plotting_matrix.columns.str.replace('^Cancer_', '', regex=True)

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="RdBu_r", figsize=(24, 12), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.0, 0.0, 0.0, 0.0), vmin=-3, vmax=3)

plt.show()

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]

#"carcinoma" related pathways

top_common_features = [
    "Cancer_REACTOME_MET_ACTIVATES_PTK2_SIGNALING",
    "Cancer_PID_EPHRINB_REV_PATHWAY",
    "Cancer_REACTOME_CONSTITUTIVE_SIGNALING_BY_ABERRANT_PI3K_IN_CANCER",
    "Cancer_KEGG_MEDICUS_REFERENCE_ITGA_B_FAK_RAC_SIGNALING_PATHWAY",
    "Cancer_PID_INTEGRIN5_PATHWAY",
     "Cancer_KEGG_PYRIMIDINE_METABOLISM",
    "Cancer_REACTOME_DETOXIFICATION_OF_REACTIVE_OXYGEN_SPECIES",
    "Cancer_WP_ELECTRON_TRANSPORT_CHAIN_OXPHOS_SYSTEM_IN_MITOCHONDRIA",
    "Cancer_PID_TOLL_ENDOGENOUS_PATHWAY",
    "Cancer_WP_P53_TRANSCRIPTIONAL_GENE_NETWORK",
   #"Cancer_BIOCARTA_VEGF_PATHWAY",
    "Cancer_REACTOME_GLUTATHIONE_CONJUGATION",
      #"Cancer_KEGG_VEGF_SignalingPATHWAY",
    "Cancer_REACTOME_SYNTHESIS_OF_BILE_ACIDS_AND_BILE_SALTS",
    "Cancer_KEGG_STEROID_HORMONE_BIOSYNTHESIS",
    "Cancer_REACTOME_SIGNALING_BY_CSF3_G_CSF",
    #"Cancer_REACTOME_PEPTIDE_HORMONE_METABOLISM",
    
]

#top_common_features = list(set(top_common_features))


cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 2000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/nonTumorRef_GSEA"
r2_folder = "Pval/"
pval_folder = "Pval/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Set p-value threshold
p_value_threshold = 0.25


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 0, combined_r2[col])



# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ['Goveia_Carmeliet_2020_patient_45',
 'Goveia_Carmeliet_2020_patient_46',
 'Goveia_Carmeliet_2020_patient_50',
 'He_Fan_2021_P1',
 'He_Fan_2021_P2',
 'He_Fan_2021_P3',
 'He_Fan_2021_P4',
 'He_Fan_2021_P5',
 'Kim_Lee_2020_P0006',
 'Kim_Lee_2020_P0008',
 'Kim_Lee_2020_P0009',
 'Kim_Lee_2020_P0018',
 'Kim_Lee_2020_P0019',
 'Kim_Lee_2020_P0020',
 'Kim_Lee_2020_P0025',
 'Kim_Lee_2020_P0028',
 'Kim_Lee_2020_P0030',
 'Kim_Lee_2020_P0031',
 'Kim_Lee_2020_P0034',
 'Kim_Lee_2020_P1006',
 'Kim_Lee_2020_P1028',
 'Kim_Lee_2020_P1049',
 'Kim_Lee_2020_P1058',
 'Lambrechts_Thienpont_2018_6149v2_3',
 'Lambrechts_Thienpont_2018_6149v2_4',
 'Lambrechts_Thienpont_2018_6653_6',
 'Laughney_Massague_2020_LX653',
 'Laughney_Massague_2020_LX661',
 'Laughney_Massague_2020_LX675',
 'Laughney_Massague_2020_LX676',
 'Laughney_Massague_2020_LX679',
 'Laughney_Massague_2020_LX680',
 'Laughney_Massague_2020_LX682',
 'Laughney_Massague_2020_LX684',
 'Leader_Merad_2021_338',
 'Leader_Merad_2021_370',
 'Leader_Merad_2021_378',
 'Leader_Merad_2021_393',
 'Leader_Merad_2021_403',
 'Leader_Merad_2021_408',
 'Leader_Merad_2021_410',
 'Leader_Merad_2021_514',
 'Leader_Merad_2021_558',
 'Leader_Merad_2021_564',
 'Leader_Merad_2021_569',
 'Leader_Merad_2021_571',
 'Leader_Merad_2021_572',
 'Leader_Merad_2021_578',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_729',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_593',
 'Leader_Merad_2021_596',
 'Leader_Merad_2021_626',
 'Leader_Merad_2021_630',
 'Leader_Merad_2021_695',
 'Leader_Merad_2021_458',
 'Leader_Merad_2021_460',
 'Leader_Merad_2021_464',
 'Leader_Merad_2021_532',
 'Leader_Merad_2021_725',
 'Leader_Merad_2021_800',
 'Leader_Merad_2021_695',
 'Goveia_Carmeliet_2020_patient_32',
 'Goveia_Carmeliet_2020_patient_40',
 'Goveia_Carmeliet_2020_patient_41',
 'Goveia_Carmeliet_2020_patient_42',
 'Lambrechts_Thienpont_2018_6149v1_1',
 'Lambrechts_Thienpont_2018_6149v1_2',
 'Lambrechts_Thienpont_2018_6653_7',
 'Leader_Merad_2021_377',
 'Leader_Merad_2021_522',
 'Leader_Merad_2021_714',
 'Leader_Merad_2021_584',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_522'
       ]

custom_order = [sample for sample in custom_order if sample in plotting_matrix.index]


# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

plotting_matrix.columns = plotting_matrix.columns.str.replace('^Cancer_', '', regex=True)

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="RdBu_r", figsize=(24, 12), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=-3, vmax=3)

plt.show()

In [ ]:
nan_counts_per_sample1

In [ ]:
import pandas as pd
import os

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_CancerRef_Primary/"
r2_folder = "R2/"
pval_folder = "Padj/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

nan_threshold = 800

metadat = metadata

# Define the cell types and the base paths
#cell_types = ["Cancer", "Stromal", "Endothelial"]
cell_types = ["Cancer"]


# Function to read and process data for samples based on GFR threshold
def process_condition(metadat, unique_samples):

    
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to count significance
def count_significance(pval_df, threshold=0.05):
    sig_count = (pval_df < threshold).sum(axis=0, skipna=True)
    nonsig_count = (pval_df >= threshold).sum(axis=0, skipna=True)
    return sig_count, nonsig_count

# Read metadata

#metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]['patient'].unique()

# Process each condition based on GFR threshold
#combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD")]["patient"].tolist())  # Samples above the threshold
#combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC")]["patient"].tolist())  # Samples below the threshold

combined_pval_cond1 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUAD") & (metadata['tissue: tumor_primary'] == 1) & (metadata['uicc_stage'].isin(["II", "III", "IV"]))]["patient"].tolist())  
combined_pval_cond2 = process_condition(metadata, metadata[(metadata['platform'] == "10x") & (metadata['condition'] == "LUSC") & (metadata['tissue: tumor_primary'] == 1)]["patient"].tolist())  


#filter out the samples with too many nan
# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample1 = combined_pval_cond1.isna().sum(axis=1)
nan_counts_per_sample2 = combined_pval_cond2.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude1 = nan_counts_per_sample1[nan_counts_per_sample1 > nan_threshold].index
samples_to_exclude2 = nan_counts_per_sample2[nan_counts_per_sample2 > nan_threshold].index

combined_pval_cond1 = combined_pval_cond1.drop(samples_to_exclude1, axis=0)
combined_pval_cond2 = combined_pval_cond2.drop(samples_to_exclude2, axis=0)


# Count significance for each condition
sig_count_cond1, nonsig_count_cond1 = count_significance(combined_pval_cond1)
sig_count_cond2, nonsig_count_cond2 = count_significance(combined_pval_cond2)

# Creating the comparison DataFrame
comparison_df = pd.DataFrame(columns=combined_pval_cond1.columns, index=['cond1_sig', 'cond1_nonsig', 'cond2_sig', 'cond2_nonsig'])
comparison_df.loc['cond1_sig'] = sig_count_cond1
comparison_df.loc['cond1_nonsig'] = nonsig_count_cond1
comparison_df.loc['cond2_sig'] = sig_count_cond2
comparison_df.loc['cond2_nonsig'] = nonsig_count_cond2


import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

# Assuming the comparison_df has been generated as in the previous code

fisher_results = pd.DataFrame(columns=['Feature', 'Test_statistic', 'p_value', "odds_ratio", 'cond1_sig_count', 'cond2_sig_count'])

for feature in comparison_df.columns:
    # Creating the contingency table for each feature
    contingency_table = comparison_df[[feature]].values.reshape(2, 2)
    
    # Extract counts of significant samples for each condition for the current feature
    cond1_sig_count = comparison_df.loc['cond1_sig', feature]
    cond2_sig_count = comparison_df.loc['cond2_sig', feature]

    # Apply Fisher's Exact Test
    odds_ratio, p = fisher_exact(contingency_table)
    
    # Store the results
    result_row = pd.DataFrame([{
        'Feature': feature, 
        'Test_statistic': odds_ratio, 
        'p_value': p, 
        "odds_ratio": odds_ratio, 
        'cond1_sig_count': cond1_sig_count,
        'cond2_sig_count': cond2_sig_count
    }])
    
    fisher_results = pd.concat([fisher_results, result_row], ignore_index=True)

# Sort the results by p-value in ascending order
sorted_results = fisher_results.sort_values(by='p_value')

# Display the most significant features
print(sorted_results.head())  # Adjust the number inside head() as needed

from statsmodels.stats.multitest import multipletests
# Extract the original p-values
p_values = sorted_results['p_value'].values

# Apply multiple test correction (using Benjamini-Hochberg method as an example)
correction_method = 'fdr_bh'  # You can change this to 'bonferroni', 'holm', etc., based on your requirement
rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method=correction_method)

# Add the corrected p-values to your DataFrame
sorted_results['corrected_p_value'] = corrected_p_values
sorted_results['rejected_h0'] = rejected

# Now sorted_results has an additional column with corrected p-values
print(sorted_results.head())  # Adjust the number inside head() as needed

sorted_results.head(100)

In [ ]:
sorted_results["Feature"].head(20)

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]

#"carcinoma" related pathways

top_common_features = [
    #"Cancer_REACTOME_BINDING_AND_UPTAKE_OF_LIGANDS_BY_SCAVENGER_RECEPTORS",
   #"Cancer_REACTOME_CHEMOKINE_RECEPTORS_BIND_CHEMOKINES",
    "Cancer_WP_PROSTAGLANDIN_SYNTHESIS_AND_REGULATION",
    "Cancer_WP_OXIDATIVE_STRESS_RESPONSE",
    "Cancer_KEGG_STEROID_HORMONE_BIOSYNTHESIS",
    "Cancer_REACTOME_BILE_ACID_AND_BILE_SALT_METABOLISM", 
    #"Cancer_PID_REG_GR_PATHWAY",
    "Cancer_KEGG_GLYCOLYSIS_GLUCONEOGENESIS",

    #"Cancer_KEGG_TOLL_LIKE_RECEPTOR_SIGNALING_PATHWAY",
    #"Cancer_BIOCARTA_IGF1_PATHWAY",
    #"Cancer_BIOCARTA_EGF_PATHWAY",
    

    
    
]

#top_common_features = list(set(top_common_features))


cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 800  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/output_CancerRef_Primary"
r2_folder = "R2/"
pval_folder = "Padj/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)




import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1
p_value_threshold = 0.05
for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ['Goveia_Carmeliet_2020_patient_45',
 'Goveia_Carmeliet_2020_patient_50',
 'He_Fan_2021_P2',
 'He_Fan_2021_P5',
 'Kim_Lee_2020_P0009',
 'Kim_Lee_2020_P0028',
 'Kim_Lee_2020_P0031',
 'Kim_Lee_2020_P1006',
 'Kim_Lee_2020_P1028',
 'Kim_Lee_2020_P1049',
 'Kim_Lee_2020_P1058',
 'Lambrechts_Thienpont_2018_6149v2_3',
 'Lambrechts_Thienpont_2018_6149v2_4',
 'Lambrechts_Thienpont_2018_6653_6',
 'Laughney_Massague_2020_LX675',
 'Laughney_Massague_2020_LX679',
 'Leader_Merad_2021_370',
 'Leader_Merad_2021_403',
 'Leader_Merad_2021_408',
 'Leader_Merad_2021_514',
 'Leader_Merad_2021_558',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_729',
 'Leader_Merad_2021_581',
 'Leader_Merad_2021_695',
 'Leader_Merad_2021_695',

'Goveia_Carmeliet_2020_patient_32',
 'Goveia_Carmeliet_2020_patient_40',
 'Goveia_Carmeliet_2020_patient_41',
 'Goveia_Carmeliet_2020_patient_42',
 'Lambrechts_Thienpont_2018_6149v1_1',
 'Lambrechts_Thienpont_2018_6149v1_2',
 'Lambrechts_Thienpont_2018_6653_7',
 'Leader_Merad_2021_377',
 'Leader_Merad_2021_522',
 'Leader_Merad_2021_714',
 'Leader_Merad_2021_584',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_706',
 'Leader_Merad_2021_522'
               
               ]
       

# Filter custom_order to keep only those elements that are in plotting_matrix's index
custom_order_filtered = [item for item in custom_order if item in plotting_matrix.index]

# Reorder the rows using the filtered list
plotting_matrix = plotting_matrix.loc[custom_order_filtered]


plotting_matrix.columns = plotting_matrix.columns.str.replace('^Cancer_', '', regex=True)

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(12, 15), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
# Define cell types and file paths
#cell_types = ["Cancer", "Epithelial", "Immune", "Stromal", "Endothelial"]

#"carcinoma" related pathways

#top_common_features = list(set(top_common_features))


cell_types = ["Cancer"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 2000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/lungatlas_clean/CancerRef_GSEA"
r2_folder = "Pval/"
pval_folder = "Pval/"
metadat_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/lungatlas/LC_sample_metadata.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x")]['patient'].unique()
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)




import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 0
p_value_threshold = 0.05
for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 0, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels

       

# Filter custom_order to keep only those elements that are in plotting_matrix's index
custom_order_filtered = [item for item in custom_order if item in plotting_matrix.index]

# Reorder the rows using the filtered list
plotting_matrix = plotting_matrix.loc[custom_order_filtered]


plotting_matrix.columns = plotting_matrix.columns.str.replace('^Cancer_', '', regex=True)

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="RdBu_r", figsize=(12, 15), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.05, 0.05, 0.10), vmin = -3, vmax = 3)

plt.show()